<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/product_specific_dbextract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,116 kB]
Hit:12 http://ppa.launchpad.net/cra

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Create dataframe with raw data from 'furnitures' category
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [6]:
# Filter the data and create a new DataFrame or table to retrieve all the rows for that specific product
df_product = df.filter(df.product_id == "B000M0MJU2")
df_product.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   46387114|R26RZ3C5VL3H5W|B000M0MJU2|     805416447|Intex Raised Down...|        Outdoors|          5|            0|          0|   N|                Y|          Five Stars|Very comfortable ...| 2015-08-31|
|         US|   44581842|R2A498KG3CWVC3|B000M0MJU2|     805416447|Intex Raised Down...|        Outdoors|          1|    

In [7]:
# Convert Pyspark dataframe into panda df
df_product_panda = df_product.toPandas()
df_product_panda

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,46387114,R26RZ3C5VL3H5W,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,5,0,0,N,Y,Five Stars,Very comfortable and easy to set up.,2015-08-31
1,US,44581842,R2A498KG3CWVC3,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,1,1,1,N,N,Cannot Recommend,This airmatress does not hold air. The first n...,2015-08-31
2,US,32473989,R33Z46RRJXS3O7,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,5,0,0,N,Y,Five Stars,Good product Great quality,2015-08-31
3,US,7668480,R1W6FG4HPA0K6C,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,5,0,0,N,Y,Five Stars,Super comfortable!!! I am 6 months pregnant an...,2015-08-31
4,US,2368997,R3JTF0SET2BV4Y,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,5,0,0,N,Y,Five Stars,good,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4748,US,10221632,R2K0TL4VNH71AB,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,5,7,8,N,Y,Intex Raised Downy Queen Airbed with Built-in ...,Good buy. Very Little Leak. The bed itself is ...,2008-06-04
4749,US,52922779,R3FMSB7NVM64SP,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,5,4,4,N,Y,great traveling bed,this is a great thing to have. the built in pu...,2008-04-12
4750,US,45643821,R3JPX7JXE4JF61,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,5,19,20,N,N,Amazing! Like a real bed!,This air mattress is incredible. We bought it...,2008-04-08
4751,US,14464916,R2XZEVT9OE0BXI,B000M0MJU2,805416447,Intex Raised Downy Airbed with Built-in Electr...,Outdoors,4,6,7,N,Y,Good Bed,I have not had this item for very long and hav...,2008-03-26


In [8]:
# Download to csv to use jupyter notebooks 
from google.colab import drive
drive.mount('/drive', force_remount=True)
df_product_panda.to_csv('/drive/My Drive/df_product.csv')

Mounted at /drive
